# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [31]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [32]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=felipe password=Jwoa0RGD")
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')
song_files

['/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBLU128F93349CF.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBAM128F429D223.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBTA128F933D304.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBOP128F931B50D.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBKX128F4285205.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBJE12903CDB442.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBZN12903CD9297.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBNP128F932546F.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBBV128F42967D7.json',
 '/home/felipe/Documents/Udacity/project1/data/song_data/A/B/B/TRABBXU128F92FEF48.json',
 '/home/felipe/Docume

In [7]:
filepath = song_files[-1]

In [8]:
df = pd.read_json(filepath, typ='series')
df

num_songs                                           1
artist_id                          ARNTLGG11E2835DDB9
artist_latitude                                  None
artist_longitude                                 None
artist_location                                      
artist_name                                       Clp
song_id                            SOUDSGM12AC9618304
title               Insatiable (Instrumental Version)
duration                                    266.39628
year                                                0
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
df2 = df.loc[["song_id", "title", "artist_id", "year", "duration"]]
df2 = df2.to_numpy()
song_data = df2.tolist()
song_data

['SOUDSGM12AC9618304',
 'Insatiable (Instrumental Version)',
 'ARNTLGG11E2835DDB9',
 0,
 266.39628]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [30]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_data = df.loc[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]]
artist_data = artist_data.to_numpy().tolist()
artist_data

['ARNTLGG11E2835DDB9', 'Clp', '', None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
log_files = get_files('data/log_data')
log_files

['/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-20-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-19-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-16-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-24-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-21-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-23-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-29-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-12-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-22-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-02-events.json',
 '/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-27-events.json',
 '/home/felipe/Docume

In [14]:
filepath = log_files[-1]
filepath

'/home/felipe/Documents/Udacity/project1/data/log_data/2018/11/2018-11-28-events.json'

In [15]:
df3 = pd.read_json(filepath, typ='series', lines=True)
df3

0      {'artist': 'Mitch Ryder & The Detroit Wheels',...
1      {'artist': 'The Spill Canvas', 'auth': 'Logged...
2      {'artist': 'Mogwai', 'auth': 'Logged In', 'fir...
3      {'artist': 'Spor', 'auth': 'Logged In', 'first...
4      {'artist': 'DJ Dizzy', 'auth': 'Logged In', 'f...
                             ...                        
428    {'artist': 'Aventura', 'auth': 'Logged In', 'f...
429    {'artist': 'Die Sekte', 'auth': 'Logged In', '...
430    {'artist': 'Kid Dynamite', 'auth': 'Logged In'...
431    {'artist': 'Eminem / Bizarre', 'auth': 'Logged...
432    {'artist': 'Explosions In The Sky', 'auth': 'L...
Length: 433, dtype: object

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
for index, row in df3.iteritems():
    if row["page"].lower() != "nextsong":
        df3.drop(labels=index, axis =0, inplace=True)
df3.dropna(inplace=True)
df3

0      {'artist': 'Mitch Ryder & The Detroit Wheels',...
1      {'artist': 'The Spill Canvas', 'auth': 'Logged...
2      {'artist': 'Mogwai', 'auth': 'Logged In', 'fir...
3      {'artist': 'Spor', 'auth': 'Logged In', 'first...
4      {'artist': 'DJ Dizzy', 'auth': 'Logged In', 'f...
                             ...                        
428    {'artist': 'Aventura', 'auth': 'Logged In', 'f...
429    {'artist': 'Die Sekte', 'auth': 'Logged In', '...
430    {'artist': 'Kid Dynamite', 'auth': 'Logged In'...
431    {'artist': 'Eminem / Bizarre', 'auth': 'Logged...
432    {'artist': 'Explosions In The Sky', 'auth': 'L...
Length: 363, dtype: object

In [17]:
t = []
for index, row in df3.iteritems():
    t.append(row["ts"])
td = pd.DataFrame (list(zip(t, t)),columns = ["time", "timestamp"])
td["time"] = pd.to_datetime(td["time"])
print(td["timestamp"])

0      1543363215796
1      1543363420796
2      1543363778796
3      1543364349796
4      1543364729796
           ...      
358    1543449016796
359    1543449264796
360    1543449294796
361    1543449413796
362    1543449470796
Name: timestamp, Length: 363, dtype: int64


In [18]:
time_data = [td["timestamp"], td["time"].dt.hour, td["time"].dt.day, td["time"].dt.isocalendar().week, td["time"].dt.month, 
            td["time"].dt.year, td["time"].dt.weekday]
print(time_data)
column_labels = ("timestamp", "hour", "day", "week", "month", "year", "weekday")
print(column_labels)

[0      1543363215796
1      1543363420796
2      1543363778796
3      1543364349796
4      1543364729796
           ...      
358    1543449016796
359    1543449264796
360    1543449294796
361    1543449413796
362    1543449470796
Name: timestamp, Length: 363, dtype: int64, 0      0
1      0
2      0
3      0
4      0
      ..
358    0
359    0
360    0
361    0
362    0
Name: time, Length: 363, dtype: int64, 0      1
1      1
2      1
3      1
4      1
      ..
358    1
359    1
360    1
361    1
362    1
Name: time, Length: 363, dtype: int64, 0      1
1      1
2      1
3      1
4      1
      ..
358    1
359    1
360    1
361    1
362    1
Name: week, Length: 363, dtype: UInt32, 0      1
1      1
2      1
3      1
4      1
      ..
358    1
359    1
360    1
361    1
362    1
Name: time, Length: 363, dtype: int64, 0      1970
1      1970
2      1970
3      1970
4      1970
       ... 
358    1970
359    1970
360    1970
361    1970
362    1970
Name: time, Length: 363, dtype: int64, 

In [19]:
time_df = pd.DataFrame(list(zip(time_data[0], time_data[1], time_data[2], time_data[3],
                           time_data[4], time_data[5], time_data[6])), columns = column_labels)
time_df

,timestamp,hour,day,week,month,year,weekday
0,1543363215796,0,1,1,1,1970,3
1,1543363420796,0,1,1,1,1970,3
2,1543363778796,0,1,1,1,1970,3
3,1543364349796,0,1,1,1,1970,3
4,1543364729796,0,1,1,1,1970,3
...,...,...,...,...,...,...,...
358,1543449016796,0,1,1,1,1970,3
359,1543449264796,0,1,1,1,1970,3
360,1543449294796,0,1,1,1,1970,3
361,1543449413796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [20]:
for i, row in time_df.iterrows():
    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

[1543363215796, 0, 1, 1, 1, 1970, 3]
[1543363420796, 0, 1, 1, 1, 1970, 3]
[1543363778796, 0, 1, 1, 1, 1970, 3]
[1543364349796, 0, 1, 1, 1, 1970, 3]
[1543364729796, 0, 1, 1, 1, 1970, 3]
[1543364950796, 0, 1, 1, 1, 1970, 3]
[1543369249796, 0, 1, 1, 1, 1970, 3]
[1543369456796, 0, 1, 1, 1, 1970, 3]
[1543377306796, 0, 1, 1, 1, 1970, 3]
[1543382018796, 0, 1, 1, 1, 1970, 3]
[1543386015796, 0, 1, 1, 1, 1970, 3]
[1543386234796, 0, 1, 1, 1, 1970, 3]
[1543389090796, 0, 1, 1, 1, 1970, 3]
[1543389321796, 0, 1, 1, 1, 1970, 3]
[1543389451796, 0, 1, 1, 1, 1970, 3]
[1543389593796, 0, 1, 1, 1, 1970, 3]
[1543389650796, 0, 1, 1, 1, 1970, 3]
[1543389913796, 0, 1, 1, 1, 1970, 3]
[1543389920796, 0, 1, 1, 1, 1970, 3]
[1543390182796, 0, 1, 1, 1, 1970, 3]
[1543390381796, 0, 1, 1, 1, 1970, 3]
[1543390549796, 0, 1, 1, 1, 1970, 3]
[1543390748796, 0, 1, 1, 1, 1970, 3]
[1543390901796, 0, 1, 1, 1, 1970, 3]
[1543391139796, 0, 1, 1, 1, 1970, 3]
[1543391350796, 0, 1, 1, 1, 1970, 3]
[1543391547796, 0, 1, 1, 1, 1970, 3]
[

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:
user_df = pd.read_json(filepath, typ='series', lines=True)

t = []
for index, row in user_df.iteritems():
    s = []
    if row["userId"] == "":
        row["userId"] = None
    s.append(row["userId"])
    s.append(row["firstName"])
    s.append(row["lastName"])
    s.append(row["gender"])
    s.append(row["level"])
    t.append(s)

    user_df = pd.DataFrame (t, columns = ["userId", "firstName", "lastName", "gender", "level"]) 
user_df

,userId,firstName,lastName,gender,level
0,80,Tegan,Levine,F,paid
1,80,Tegan,Levine,F,paid
2,80,Tegan,Levine,F,paid
3,80,Tegan,Levine,F,paid
4,80,Tegan,Levine,F,paid
...,...,...,...,...,...
428,73,Jacob,Klein,M,paid
429,24,Layla,Griffin,F,paid
430,73,Jacob,Klein,M,paid
431,73,Jacob,Klein,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
conn.commit()


for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [47]:
for index, row in df3.iteritems():
    s = []
    s.append(index)
    s.append(row["ts"])
    s.append(int(row["userId"]))
    s.append(row["level"])

    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row["song"], row["artist"], row["length"]))
    results = cur.fetchone()
    
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
        
    s.append(songid)
    s.append(artistid)
    s.append(row["sessionId"])
    s.append(row["location"])
    s.append(row["userAgent"])
    
    print(s)
    cur.execute(songplay_table_insert, s)
    conn.commit()

None
[0, 1543363215796, 80, 'paid', None, None, 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[1, 1543363420796, 80, 'paid', None, None, 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[2, 1543363778796, 80, 'paid', None, None, 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[3, 1543364349796, 80, 'paid', None, None, 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[4, 1543364729796, 80, 'paid', None, None, 992, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, 

None
[283, 1543430935796, 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[284, 1543431033796, 88, 'paid', None, None, 999, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[285, 1543431151796, 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
None
[286, 1543431301796, 88, 'paid', None, None, 999, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"']
None
[287, 1543431430796, 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHT

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [48]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.